# TODO: 概要

# ライブラリのインポート

In [29]:
import os
import json
import datetime

from dotenv import load_dotenv, find_dotenv

from semantic_kernel.agents import (
    ChatCompletionAgent,
    ChatHistoryAgentThread,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.contents import (
    FunctionCallContent,
    FunctionResultContent,
    TextContent,
)


# 環境変数の取得

In [30]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

# ユーティリティ関数

In [31]:
async def print_thread_message_details(thread: str):
    """
    スレッドのメッセージ詳細を表示します。

    Args:
        thread (str): スレッドのインスタンス
    """
    async for message in thread.get_messages():
        print("-----")

        for item in message.items:
            if isinstance(item, FunctionCallContent):
                print(f"[Function Calling] by {message.ai_model_id}")
                print(f" - Function Name : {item.name}")
                print(f" - Arguments     : {item.arguments}")

            elif isinstance(item, FunctionResultContent):
                print(f"[Function Result]")
                # 文字列のデコード変換
                if isinstance(item.result, str):
                    try:
                        decoded = json.loads(item.result)
                        print(f" - Result        : {decoded}") # デコード成功時は変換後の値を表示
                    except json.JSONDecodeError:
                        print(f" - Result        : {item.result}")  # デコード失敗時はそのまま
                else:
                    print(f" - Result        : {item.result}")

            elif isinstance(item, TextContent):
                if message.name:
                    print(f"[Agent Response] from {message.ai_model_id}")
                else:
                    print("[User Message]")
                print(f" - Content       : {item.text}")

            else:
                print(f"[Unknown Item Type] ({type(item).__name__})")
                print(f" - Raw Item      : {item}")


def log_with_timestamp(message: str) -> None:
    """
    現在時刻付きでメッセージを標準出力にログとして表示します。

    Args:
        message (str): 出力するログメッセージ。
    """
    timestamp = datetime.datetime.now().strftime("%H:%M:%S.%f")[:-3]
    print(f"[{timestamp}] {message}")

# プラグインの作成

In [32]:
# MCPのエンドポイントURL
MCP_URL = "http://127.0.0.1:8000/mcp/"


mcp_plugin = MCPStreamableHttpPlugin(
    name="mcp_plugin",
    url=MCP_URL
)

await mcp_plugin.connect()

# クライアントの初期化

In [33]:
# Chat Completion API クライアントの初期化
azure_completion_service  = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

In [34]:
# Chat Completion Agent クライアントの初期化
agent = ChatCompletionAgent(
    service=azure_completion_service,
    name="mcp_agent",
    instructions=(
        "あなたはECサイトの分析エージェントです。"
        "PostgreSQLの各種マスター、注文、ユーザー情報と、CosmosDBに格納されたレビュー分析データを、mcp_pluginとして利用できます。"
    ),
    plugins=[mcp_plugin],
)

## スレッドの作成

In [35]:
# Thread の作成
thread = ChatHistoryAgentThread()
print(f"Created Thread. THREAD_ID: {thread.id}")


Created Thread. THREAD_ID: thread_d9715ea449414838a8674efbcd788142


## レスポンスを取得

In [36]:
# response = await agent.get_response(
#     messages=["2024年12月は何がどれくらい購入されましたか？"],
#     thread=thread,
# )

# print(response)

In [37]:
# await print_thread_message_details(thread)

In [38]:
response = await agent.get_response(
    messages=["2024年1月～12月の月別売上を教えてください"],
    thread=thread,
)

print(response)

2024年1月～12月の月別売上（主要カテゴリ合算）は以下のとおりです。

| 月   | 売上合計（円）         | 備考（主要カテゴリ）                        |
|------|-----------------------|---------------------------------------|
| 1月  | 728,500               | PC・タブレット、ソフトウェア、アクセサリ、ゲームエンタメなど            |
| 2月  | 641,000               | PC・タブレット、アクセサリ、ゲームエンタメ            |
| 3月  | 441,000               | PC・タブレット、アクセサリ、ソフトウェア                |
| 4月  | 8,891,000             | PC・タブレットが非常に高額（約868万円）                   |
| 5月  | 750,000               | PC・タブレット、アクセサリ                        |
| 6月  | 385,500               | アクセサリ、PC・タブレット、ゲームエンタメ              |
| 7月  | 44,000                | アクセサリのみ                                    |
| 8月  | 821,500               | PC・タブレット、アクセサリ、ゲームエンタメ                |
| 9月  | 432,000               | PC・タブレット、ソフトウェア、アクセサリ                |
| 10月 | 417,000               | PC・タブレット、アクセサリ                            |
| 11月 | 189,000               | ゲームエンタメ、アクセサリ                            |
| 12月 | 1,776,000      

In [ ]:
response = await agent.get_response(
    messages=["4月の商品別販売額を教えてください"],
    thread=thread,
)

print(response)

2024年4月の商品別販売額（上位）は以下の通りです。

- Surface Laptop：8,500,000円
- Surface Go：180,000円
- Type Cover：40,000円
- USB-C トラベルハブ：33,000円
- Surface ペン：16,000円
- Microsoft 365：16,000円
- Arc Mouse：11,000円
- Xbox Game Pass：4,500円

金額ベースの販売集計です。「販売個数」のご要望の場合は、商品の単価情報等が別途必要となるため、詳細が欲しい場合はご指示ください。


In [40]:
response = await agent.get_response(
    messages=["4月の商品別販売個数を教えてください"],
    thread=thread,
)

print(response)

2024年4月の商品別販売個数は以下の通りです。（販売額 ÷ 単価で計算）

- Surface Laptop：8,500,000円 ÷ 170,000円 = 50台
- Surface Go：180,000円 ÷ 90,000円 = 2台
- Type Cover：40,000円 ÷ 20,000円 = 2個
- USB-C トラベルハブ：33,000円 ÷ 11,000円 = 3個
- Surface ペン：16,000円 ÷ 16,000円 = 1本
- Microsoft 365：16,000円 ÷ 16,000円 = 1本
- Arc Mouse：11,000円 ÷ 11,000円 = 1個
- Xbox Game Pass：4,500円 ÷ 1,500円 = 3本

合計個数やほかの商品についても知りたい場合はお知らせください。


In [41]:
response = await agent.get_response(
    messages=["Surface Laptopの2024年1月から12月の月別販売数量を教えてください"],
    thread=thread,
)

print(response)

Surface Laptop（単価：170,000円）の2024年1月～12月月別販売数量は以下の通りです。

| 月   | 販売金額 | 販売数量（台） |
|------|---------|---------------|
| 1月  | 510,000 | 3台           |
| 2月  | -       | 0台           |
| 3月  | 170,000 | 1台           |
| 4月  | 8,500,000 | 50台         |
| 5月  | -       | 0台           |
| 6月  | -       | 0台           |
| 7月  | -       | 0台           |
| 8月  | 170,000 | 1台           |
| 9月  | 340,000 | 2台           |
| 10月 | -       | 0台           |
| 11月 | -       | 0台           |
| 12月 | 170,000 | 1台           |

「-」はその月の販売記録がない（＝0台）ことを示します。
他商品やさらに詳細がご入り用の場合もご相談ください。


In [42]:
response = await agent.get_response(
    messages=["Surface Laptopの4月の商品レビューには何が書かれていますか？"],
    thread=thread,
)

print(response)

2024年4月の「Surface Laptop」に関する商品レビューには、ポジティブ・ネガティブ含めてテキストコメントの投稿がありませんでした。

他の月のレビューや、別商品についてのお問い合わせも承りますので、ご希望があればお知らせください。
